In [2]:
import torch
from vfm import VFM
from scripts.torch_to_onnx import fix_onnx_fp16, convert_pytorch_to_onnx

/home/ubuntu/vlm-vfm-processing-pipeline/.venv/lib/python3.10/site-packages/onnxscript/converter.py:823: FutureWarning: 'onnxscript.values.Op.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()
/home/ubuntu/vlm-vfm-processing-pipeline/.venv/lib/python3.10/site-packages/onnxscript/converter.py:823: FutureWarning: 'onnxscript.values.OnnxFunction.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()


### Torch -> ONNX conversion

In [ ]:

device = "cpu"
dtype = torch.float32
# Instantiate the model
model = VFM(device=device, dtype=dtype)
model.eval()  # Set model to evaluation mode

all_pixel_values = torch.load(
    "/home/ubuntu/vlm-vfm-processing-pipeline/test_data/all_pixel_values.pkl",
    weights_only=True,
    map_location=device,
)
patch_attn_mask = torch.load(
    "/home/ubuntu/vlm-vfm-processing-pipeline/test_data/patch_attn_mask.pkl",
    weights_only=True,
    map_location=device,
)
tgt_sizes = torch.load(
    "/home/ubuntu/vlm-vfm-processing-pipeline/test_data/tgt_sizes.pkl",
    weights_only=True,
    map_location=device,
)

# Set dynamic axes for inputs/outputs
dynamic_axes = {
    "all_pixel_values": {0: "batch_size"},
    "patch_attn_mask": {0: "batch_size"},
    "tgt_sizes": {0: "batch_size"},
    "vision_embedding": {0: "batch_size"},
}

convert_pytorch_to_onnx(
    model=model,
    input_sample=(
        all_pixel_values,
        patch_attn_mask,
        tgt_sizes
        ),
    onnx_path="models/vfm.onnx",
    input_names=["all_pixel_values", "patch_attn_mask", "tgt_sizes"],
    output_names=["vision_embedding"],
    dynamic_axes=dynamic_axes,
    opset_version=19,  # Higher opset for newer operators
    export_params=True,  # Include model weights
    do_constant_folding=True,  # Optimize constant operations
    verbose=True,  # Show detailed conversion info
    training=torch.onnx.TrainingMode.EVAL,  # Export in inference mode
    enable_onnx_checker=True,  # Verify model structure
    optimize_onnx=False,  # Apply optimizations
)


### fix_onnx_fp16

Modify the onnx file to handle constants > FP16_Max and < FP16_Min . fix_onnx_fp16 is a helper function for this purpose.
In the exported model, -inf is represented by the min value in FP32. The helper function modifies that to min in FP16.

In [4]:
fp16_model_name = fix_onnx_fp16(
    gen_models_path="models",
    model_base_name="vfm",
)

Found constants out of FP16 range, clipped to FP16 range
Saving modified onnx file at models/vfm_fix_outofrange_fp16.onnx


### QPC Compilation step
- !This should run from inside the QC AI SDK Container
qaic-exec cli tool is used to compile the model for Qualcomm AI Cloud 100. The input to this tool is onnx file generated above. The tool produces a QPC (Qualcomm Program Container) binary file in the path defined by -aic-binary-dir argument.

Breakdown of key compile parameters.  
We have compiled the onnx file

* with 16 NSP cores
* with float 16 precision
* defined onnx symbols

In [6]:
!/opt/qti-aic/exec/qaic-exec \
    -m=/workspace/vfm_fix_outofrange_fp16.onnx \
    -aic-num-cores=16 -convert-to-fp16 \
    -onnx-define-symbol=batch_size,30 \
    -aic-binary-dir=/workspace/vfm_fix_outofrange_fp16_qpc \
    -aic-hw -aic-hw-version=2.0 -compile-only

/bin/bash: line 1: /opt/qti-aic/exec/qaic-exec: No such file or directory
